Congrats! You just graduated UVA's BSDS program and got a job working at a movie studio in Hollywood. 

Your boss is the head of the studio and wants to know if they can gain a competitive advantage by predicting new movies that might get high imdb scores (movie rating). 

You would like to be able to explain the model to mere mortals but need a fairly robust and flexible approach so you've chosen to use decision trees to get started. 

In doing so, similar to  great data scientists of the past you remembered the excellent education provided to you at UVA in a undergrad data science course and have outline 20ish steps that will need to be undertaken to complete this task. As always, you will need to make sure to #comment your work heavily. 

 Footnotes: 
-	You can add or combine steps if needed
-	Also, remember to try several methods during evaluation and always be mindful of how the model will be used in practice.
- Make sure all your variables are the correct type (factor, character,numeric, etc.)

In [55]:
import pandas as pd
import numpy as np
import os
import glob
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import export_text
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
# done to see workspace (file paths)
 
print(os.getcwd())
print(os.listdir())

# Searching for the file "movie_metadata.csv"
file_path = glob.glob('/workspaces/DS-3021/**/movie_metadata.csv', recursive=True)
print(file_path) 

/workspaces/DS-3021/08_DT_Class
['Classification and regression trees_Loh.pdf', 'clinical_breast_cleaned.csv', 'Decision Trees.ipynb', 'Decision Trees.py', 'Decision_Trees_10.20.22.pptx', 'Decision_Trees_3.24.22.pptx', 'Decision_Trees_4.28.21.pdf', 'DT_Class_Lab copy.ipynb', 'DT_Class_Lab.ipynb', 'heart_failure.csv', 'parent.csv', 'pregnancy.csv', 'Week 08 conversion doc.docx', '~$Decision_Trees_10.20.22.pptx']
['/workspaces/DS-3021/data/movie_metadata.csv']


In [58]:
# Loading the data
# Sometimes need to set the working directory back out of a folder that we create a file in

# import os
# os.listdir()
# print(os.getcwd())
# os.chdir('c:\\Users\\Brian Wright\\Documents\\3001Python\\DS-3001')

movie_metadata = pd.read_csv("../data/movie_metadata.csv")    

2 Ensure all the variables are classified correctly including the target variable and collapse factor variables as needed.

In [61]:
# Checking data types of all columns
print(movie_metadata.dtypes)

# Identifying numeric and categorical variables
numeric_columns = movie_metadata.select_dtypes(include=['number']).columns.tolist()
categorical_columns = movie_metadata.select_dtypes(include=['object']).columns.tolist()

print("\nNumeric Columns:", numeric_columns)
print("\nCategorical Columns:", categorical_columns)

# Making sure that 'imdb_score' is numeric
print("\nData type of imdb_score:", movie_metadata['imdb_score'].dtype)

color                         object
director_name                 object
num_critic_for_reviews       float64
duration                     float64
director_facebook_likes      float64
actor_3_facebook_likes       float64
actor_2_name                  object
actor_1_facebook_likes       float64
gross                        float64
genres                        object
actor_1_name                  object
movie_title                   object
num_voted_users                int64
cast_total_facebook_likes      int64
actor_3_name                  object
facenumber_in_poster         float64
plot_keywords                 object
movie_imdb_link               object
num_user_for_reviews         float64
language                      object
country                       object
content_rating                object
budget                       float64
title_year                   float64
actor_2_facebook_likes       float64
imdb_score                   float64
aspect_ratio                 float64
m

3 Check for missing variables and correct as needed. Once you've completed the cleaning again create a function that will do this for you in the future. In the submission, include only the function and the function call.

In [63]:
# Checking for missing values
missing_values = movie_metadata.isnull().sum()
print("Missing Values:\n", missing_values[missing_values > 0])

Missing Values:
 color                       19
director_name              104
num_critic_for_reviews      50
duration                    15
director_facebook_likes    104
actor_3_facebook_likes      23
actor_2_name                13
actor_1_facebook_likes       7
gross                      884
actor_1_name                 7
actor_3_name                23
facenumber_in_poster        13
plot_keywords              153
num_user_for_reviews        21
language                    14
country                      5
content_rating             303
budget                     492
title_year                 108
actor_2_facebook_likes      13
aspect_ratio               329
dtype: int64


In [65]:
# Handling missing vlaues with function
# Filling catagorical columns with their mode and numeric columns with their median

def handle_missing_values(df):
    # categorical columns with mode
    categorical_cols = df.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        df[col] = df[col].fillna(df[col].mode()[0] if df[col].mode().size > 0 else "Unknown")
    
    # numeric columns with median
    numeric_cols = df.select_dtypes(include=['number']).columns
    for col in numeric_cols:
        df[col] = df[col].fillna(df[col].median())
    
    return df

movie_metadata = handle_missing_values(movie_metadata)

# Making sure that there are no missing values
print("Missing values after cleaning:\n", movie_metadata.isnull().sum().sum())


Missing values after cleaning:
 0


4 Guess what, you don't need to scale the data, because DTs don't require this to be done, they make local greedy decisions...keeps getting easier, go to the next step.

5 Determine the baserate or prevalence for the classifier, what does this number mean?

In [68]:
# Creating a binary target based on imdb_score 
movie_metadata['high_imdb_score'] = movie_metadata['imdb_score'] >= 7

# Calculating prevalence or the baserate of high IMDb score
prevalence = movie_metadata['high_imdb_score'].mean()
print(f"Prevalence of high IMDb scores (7 or higher): {prevalence:.4f}")

Prevalence of high IMDb scores (7 or higher): 0.3526


The prevalence is the proportion of instances where the target variable (e.g., "high IMDb score") occurs in the dataset. The prevalence is 0.3 meaning 30% of the movies have a high IMDb score (7 or above).

6 Split your data into test, tune, and train. (80/10/10)

In [69]:
# Define the target column 
target_column = 'high_imdb_score'  

# Separate features (X) and labels (y)
X = movie_metadata.drop(columns=[target_column])  # Drop the target column to get features
y = movie_metadata[target_column]  

# Splitting into train (80%) and then remaining into test and tune (20%) 
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)

# Splitting the remaining 20% into test (50%) and tune (50%) and giving 10% for both
X_test, X_tune, y_test, y_tune = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Looking over the sizes of the splits
print(f"Train set size: {len(X_train)}")
print(f"Tune set size: {len(X_tune)}")
print(f"Test set size: {len(X_test)}")



Train set size: 4034
Tune set size: 505
Test set size: 504


7 Create the kfold object for cross validation.

In [72]:
# Setting the number of folds 
k = 5

# Creating the KFold object which will be used for cross-validation 
kfold = KFold(n_splits=k, shuffle=True, random_state=42)

# Examples to check 
for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train, y_train), 1):
    print(f"Fold {fold}:")
    print(f"Train indices: {train_idx[:5]}...") 
    print(f"Validation indices: {val_idx[:5]}...")  
    print("-" * 40)


Fold 1:
Train indices: [0 1 2 3 4]...
Validation indices: [ 6  8 12 14 17]...
----------------------------------------
Fold 2:
Train indices: [1 2 3 4 5]...
Validation indices: [ 0  7 22 31 41]...
----------------------------------------
Fold 3:
Train indices: [0 1 3 4 5]...
Validation indices: [ 2 13 18 20 37]...
----------------------------------------
Fold 4:
Train indices: [0 1 2 4 6]...
Validation indices: [ 3  5  9 10 15]...
----------------------------------------
Fold 5:
Train indices: [0 2 3 5 6]...
Validation indices: [ 1  4 11 16 19]...
----------------------------------------


8 Create the scoring metric you will use to evaluate your model and the max depth hyperparameter (grid search) 

In [73]:
# Identifying categorical columns within X_train
categorical_columns = X_train.select_dtypes(include=['object']).columns.tolist()

# Applying one-hot encoding to categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
    ],
    remainder='passthrough' 
)

# Transforming X_train and X_tune
X_train_encoded = preprocessor.fit_transform(X_train)
X_tune_encoded = preprocessor.transform(X_tune)

# Setting the parameter grid for max_depth
param_grid = {'max_depth': range(1, 21)}

# Decision Tree classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# GridSearchCV with the classifier, parameter grid, and cross-validation
grid_search = GridSearchCV(estimator=dt_classifier, param_grid=param_grid, cv=kfold, scoring='accuracy', n_jobs=-1)

# Gridsearch and output of best hyperparameter value
grid_search.fit(X_train_encoded, y_train)
print(f"Best max_depth from grid search: {grid_search.best_params_['max_depth']}")


Best max_depth from grid search: 1


9 Build the classifier object 

In [ ]:
# Creating a Decision Tree classifier object 
dt_model = DecisionTreeClassifier(max_depth=grid_search.best_params_['max_depth'], random_state=42)

10 Use the kfold object and the scoring metric to find the best hyperparameter value for max depth via the grid search method.

In [75]:
# This was already done in step 8 with GridSearchCV. 
# The grid search finds the best hyperparameter and stores it in grid_search.best_params_.

11 Fit the model to the training data.

In [76]:
# Fitting the decision tree to the whole training data with the best hyperparameter
dt_model.fit(X_train_encoded, y_train)

DecisionTreeClassifier(max_depth=1, random_state=42)

12 What is the best depth value?

In [77]:
print(f"Best max_depth: {grid_search.best_params_['max_depth']}")

Best max_depth: 1


13 Print out the model

In [79]:
# Getting feature names after one-hot encoding
encoded_feature_names = preprocessor.get_feature_names_out()

# The strucuture of the decision tree
tree_structure = export_text(dt_model, feature_names=encoded_feature_names)
print(tree_structure)


|--- remainder__imdb_score <= 6.95
|   |--- class: False
|--- remainder__imdb_score >  6.95
|   |--- class: True



14 View the results, comment on how the model performed using the metrics you selected.

In [ ]:
# Predicting on the training data using the encoded features
y_train_pred = dt_model.predict(X_train_encoded)

# Calculating accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_accuracy:.4f}")


Training Accuracy: 1.0000


In [81]:
# Confusion matrix
cm = confusion_matrix(y_train, y_train_pred)
print("Confusion Matrix:\n", cm)

Confusion Matrix:
 [[2617    0]
 [   0 1417]]


The model has a training accuracy of 1.0000, meaning it perfectly predicts the training data. However, this might mean the model is overfitting and could perform worse on new, unseen data. Therefore, checking the test set performance is important to confirm this.

15 Which variables appear to be contributing the most (variable importance) 

In [82]:
# Getting feature importances
importances = dt_model.feature_importances_

# Creating a DataFrame to see feature importance
importance_df = pd.DataFrame({
    'Feature': encoded_feature_names,
    'Importance': importances
})

# Sorting by importance
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print(importance_df)


                                                 Feature  Importance
22095                              remainder__imdb_score         1.0
0                            cat__color_ Black and White         0.0
14729  cat__plot_keywords_alien|creature|future|outer...         0.0
14737  cat__plot_keywords_alien|estranged couple|ocea...         0.0
14736  cat__plot_keywords_alien|escape|manager|neighb...         0.0
...                                                  ...         ...
7364                  cat__movie_title_A Scanner Darkly          0.0
7363                cat__movie_title_A Room with a View          0.0
7362            cat__movie_title_A Room for Romeo Brass          0.0
7361          cat__movie_title_A Prairie Home Companion          0.0
22097                    remainder__movie_facebook_likes         0.0

[22098 rows x 2 columns]


The feature remainder__imdb_score has the highest importance score of 1.0, meaning it is the most important variable in predicting the target.

16 Use the predict method on the tune data and print out the results.

In [83]:
# Transforming the tuning set using the preprocessor
X_tune_encoded = preprocessor.transform(X_tune)

# Predicting on the tuning set
y_tune_pred = dt_model.predict(X_tune_encoded)

# Printing predictions
print(f"Tuning Set Predictions:\n{y_tune_pred}")


Tuning Set Predictions:
[ True False False False False False False  True False False False False
 False False False False  True False  True  True False False False False
  True False  True False  True False False False False False False  True
  True False False  True False False False False False False False  True
 False  True  True  True False False  True  True False  True  True False
  True False False False False False False False  True  True  True  True
  True False False False False False  True False False False False  True
 False  True  True False False False  True False False  True  True False
 False  True False False  True False False  True False  True  True False
 False  True False False  True False  True False False False  True  True
 False False False False  True  True False False False  True False False
  True False False False  True  True  True False False False False False
 False False False False  True False False  True  True False False False
  True  True  True False Fa

17 How does the model perform on the tune data?

In [ ]:
# Accuracy of tuning set
tune_accuracy = accuracy_score(y_tune, y_tune_pred)
print(f"Tuning Set Accuracy: {tune_accuracy:.4f}")

# Confusion matrix for the tuning set
cm_tune = confusion_matrix(y_tune, y_tune_pred)
print("Confusion Matrix for Tuning Set:\n", cm_tune)


Tuning Set Accuracy: 1.0000
Confusion Matrix for Tuning Set:
 [[322   0]
 [  0 183]]


This indicates that the model is making perfect predictions on the tuning set, with no errors. However, might signify overfitting and is also important to check performance on test. 

18 Print out the confusion matrix for the tune data, what does it tell you about the model?

In [85]:
# Confusion matrix for tuning set
cm_tune = confusion_matrix(y_tune, y_tune_pred)
print("Confusion Matrix for Tuning Set:\n", cm_tune)


Confusion Matrix for Tuning Set:
 [[322   0]
 [  0 183]]


The model shows perfect accuracy on the test set as well and it suggests that the model is overfitting, meaning it might have memorized the data rather than learning to generalize. While it performs well on known data, it could struggle with new, unseen data.

19 What are the top 3 movies based on the tune set? Which variables are most important in predicting the top 3 movies?

In [ ]:
# Getting the top 3 movies that are predicted to have a high IMDb score
top_3_movies = movie_metadata.iloc[y_tune_pred.argsort()[-3:]]  
print(top_3_movies[['movie_title', 'imdb_score', 'high_imdb_score']])

# Checking variables most important to predict
print(importance_df.head(3))


                         movie_title  imdb_score  high_imdb_score
112                    Transformers          7.1             True
83   Dawn of the Planet of the Apes          7.6             True
0                            Avatar          7.9             True
                                                 Feature  Importance
22101                              remainder__imdb_score         1.0
0                            cat__color_ Black and White         0.0
14733  cat__plot_keywords_alien|crash|planet|prison|s...         0.0


20 Use a different hyperparameter for the grid search function and go through the process above again using the tune set. 

In [ ]:
param_grid_min_samples = {'min_samples_split': [2, 5, 10, 20]}

# Creating GridSearchCV with new hyperparameter
grid_search_min_samples = GridSearchCV(estimator=dt_model, param_grid=param_grid_min_samples, 
                                       scoring='accuracy', cv=kfold, n_jobs=-1)

# Fitting the grid search to the preprocessed data
grid_search_min_samples.fit(X_train_encoded, y_train)

# The best hyperparameter value for min_samples_split
print(f"Best min_samples_split: {grid_search_min_samples.best_params_['min_samples_split']}")


Best min_samples_split: 2


21 Did the model improve with the new hyperparameter search?

In [87]:
# Fiting the model with the new best parameter
dt_model_min_samples = DecisionTreeClassifier(min_samples_split=grid_search_min_samples.best_params_['min_samples_split'], random_state=42)
dt_model_min_samples.fit(X_train_encoded, y_train)  

# Evaluating performance on the tuning set
y_tune_pred_min_samples = dt_model_min_samples.predict(X_tune_encoded) 
tune_accuracy_min_samples = accuracy_score(y_tune, y_tune_pred_min_samples)
print(f"New Tuning Set Accuracy with min_samples_split: {tune_accuracy_min_samples:.4f}")


New Tuning Set Accuracy with min_samples_split: 1.0000


22 Using the better model, predict the test data and print out the results.

In [88]:
# Preprocessing the test set using the preprocessor
X_test_encoded = preprocessor.transform(X_test)

# Using the best model from the hyperparameter search 
y_test_pred = dt_model_min_samples.predict(X_test_encoded)

# Printing predictions for the test set
print(f"Test Set Predictions:\n{y_test_pred}")


Test Set Predictions:
[ True False False False False  True  True False False False False  True
 False  True False  True False False  True  True  True False False False
  True False False False  True False  True False  True False False False
 False False False  True False  True False  True  True False False False
 False False False False  True  True  True False False False False False
 False False  True  True False False False  True  True False False False
  True False False  True  True False  True  True  True False False  True
 False False False False False  True  True False False  True False False
 False  True False False  True False False  True  True  True  True  True
 False False False False  True  True  True False False False False False
  True  True False False False  True False False  True False False False
  True  True  True  True False False False False False False False False
 False False  True False False  True False False  True  True  True False
  True  True False False Fals

23 Summarize what you learned along the way and make recommendations to your boss on how this could be used moving forward, being careful not to over promise.

The model is doing great on the training, tuning, and test sets, but I’m concerned it might be overfitting and could have trouble with new data. I think we should test it with real-world data and try techniques like cross-validation to make it more reliable. Moving forward, I can use the model to predict movie success, but I’ll need to keep an eye on its performance to make sure it stays accurate.